# EVA Descriptor

In [21]:
import pandas as pd
from functools import reduce
from math import sqrt,pi,exp
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import find_peaks,find_peaks_cwt
import networkx as nx
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy import stats
import os

In [22]:
# ##code for mol3dfile


# parent_directory = 'C:/Users/skdha/Downloads/Olfaction_project/irchracterizationcnn--main (3)/irchracterizationcnn--main/nist_dataset/mol_3d/'
# subdirectory_names = [d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))]

# def file_generate(cas_number):
#     file_path = parent_directory+cas_number+"/downloaded_files/mol3D.cgi"
#     if not os.path.exists(file_path):
#         return None
    
#     # Replace 'cgi_file_path' with the actual path to your CGI file
#     cgi_file_path = file_path

#     # Read the file
#     with open(cgi_file_path, 'r') as file:
#         lines = file.readlines()

#     # Initialize variables to store the IR frequency data
#     ir_data = []

#     # Find the start and end lines of the IR frequency data
#     start_line = None
#     end_line = None
#     for idx, line in enumerate(lines):
#         if "> <IR.FREQUENCIES>" in line:
#             start_line = idx + 2  # Skip header line
#         elif start_line is not None and line.strip() == '':
#             end_line = idx 
#             break

#     # Extract IR frequency data
#     if start_line is not None and end_line is not None:
#         for line in lines[start_line:end_line]:
#             freq, intensity = map(float, line.strip().split())
#             ir_data.append({'Frequency': freq, 'Intensity': intensity})

#     # Create a pandas DataFrame from the extracted data
#     ir_df = pd.DataFrame(ir_data,columns=None)
#     return ir_df


In [23]:
#code for sdf file

parent_directory = 'sdf/'
sdf_files = [file for file in os.listdir(parent_directory) if file.endswith('.sdf')]

subdirectory_names= []
for i in sdf_files:
    subdirectory_names.append(i.split(".sdf")[0])
    

def file_generate(cas_number):
    file_path = parent_directory+cas_number+".sdf"
    if not os.path.exists(file_path):
        return None
    
    # Replace 'cgi_file_path' with the actual path to your CGI file
    cgi_file_path = file_path

    # Read the file
    with open(cgi_file_path, 'r') as file:
        lines = file.readlines()

    # Initialize variables to store the IR frequency data
    ir_data = []

    # Find the start and end lines of the IR frequency data
    start_line = None
    end_line = None
    for idx, line in enumerate(lines):
        if "> <IR.FREQUENCIES>" in line:
            start_line = idx + 2  # Skip header line
        elif start_line is not None and line.strip() == '':
            end_line = idx 
            break

    # Extract IR frequency data
    if start_line is not None and end_line is not None:
        for line in lines[start_line:end_line]:
            freq, intensity = map(float, line.strip().split())
            ir_data.append({'Frequency': freq, 'Intensity': intensity})

    # Create a pandas DataFrame from the extracted data
    ir_df = pd.DataFrame(ir_data,columns=None)
    return ir_df

In [24]:
feature_vectors=[]

In [25]:
cas_no = []

In [26]:
for i in range (len(subdirectory_names)):
    file_name = file_generate(subdirectory_names[i])
   
    if (not isinstance(file_name,pd.DataFrame)) or file_name.empty:
        print(subdirectory_names[i])
        continue
    cas_no.append(subdirectory_names[i])
    x_values = file_name['Frequency']
    y_values = file_name['Intensity']
    x = np.around(x_values).astype(int)
    new_x = list(range(1,4001))
    
    # Create the old DataFrame
    old_data = {'common_column': x, 'value_column': y_values}
    old_df = pd.DataFrame(old_data)

    # Create the new DataFrame with the common column values you want
    new_data = {'common_column': new_x}
    new_df = pd.DataFrame(new_data)

    # Merge the new DataFrame with the old DataFrame based on the common column
    merged_df = new_df.merge(old_df, on='common_column', how='left')

    # Fill missing values with zeros
    merged_df['value_column'] = merged_df['value_column'].fillna(0)
    
    smoothed = []

    # gaussian function
    def gaussian_kernel(x, mean, sigma):
        return np.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * np.sqrt(2 * np.pi))


    for i in merged_df['common_column'].to_list():
        kv = gaussian_kernel(merged_df['common_column'],i,10)
    #     kv /= kv.sum()
        smoothed.append((merged_df['value_column']*kv).sum())


    # # Calculate the sum of the list
    max_val = max(smoothed)

    # # Normalize the list
    normalized_list = [x / max_val for x in smoothed]
    
        # now produce features for the model
    # Reshape the array into a 2D array of shape (num_rows, 5)

    elements_to_remove = len(normalized_list) % 5

    # Use slicing to remove elements from the beginning of the list
    normalized_list = normalized_list[elements_to_remove:]

    reshaped_array = np.array(normalized_list).reshape(-1, 5)

    # Calculate the mean along the rows
    eva_des = np.mean(reshaped_array, axis=1)
    
    # Check if the list length exceeds 600
    if len(eva_des) > 4000:
        excess = len(eva_des) - 4000  # Calculate the number of excess elements

        # Trim the list by removing 'excess' elements from both the beginning and end
        trimmed_list = eva_des[excess // 2:-excess // 2]

        # Now 'trimmed_list' contains at most 600 elements
    else:
        trimmed_list = eva_des  # If the list is already 600 elements or less, no need to trim


    # trimmed list is features list
  
    feature_vectors.append(trimmed_list)
# column_names = ['CAS_Number', 'Feature2', 'Feature3', 'Feature4', 'Feature5']
   

In [27]:
print(len(feature_vectors))


1


In [35]:
df = pd.DataFrame(feature_vectors, columns=None) 

In [36]:
df.insert(loc = 0,
          column = 'cas_no',
          value = cas_no)

In [37]:
df['cas_no'].astype(int)

0    100061
Name: cas_no, dtype: int32

In [ ]:
df.to_csv("mol3d_data.csv",index=False)

In [1]:
#Feature from sdf file
import pandas as pd
import numpy as np
import os
from scipy.signal import find_peaks, find_peaks_cwt


In [ ]:
parent_directory = 'sdf/'

# List all the .sdf files in the parent directory
sdf_files = [file for file in os.listdir(parent_directory) if file.endswith('.sdf')]
cas_no = []
feature_vectors_gaussian = []
for sdf_file in sdf_files:
    cas_number = os.path.splitext(sdf_file)[0]
    file_path = os.path.join(parent_directory, sdf_file)
    with open(file_path, 'r') as file:
        lines = file.readlines()
    ir_data = []
    start_line = None
    end_line = None
    for idx, line in enumerate(lines):
        if "> <IR.FREQUENCIES>" in line:
            start_line = idx + 2 
        elif start_line is not None and line.strip() == '':
            end_line = idx
            break
    try:
        if start_line is not None and end_line is not None:
            for line in lines[start_line:end_line]:
                freq, intensity = map(float, line.strip().split())
                ir_data.append({'Frequency': freq, 'Intensity': intensity})
        ir_df = pd.DataFrame(ir_data)

        # Generate feature vector with all 4000 features
        x_values = ir_df['Frequency']
        y_values = ir_df['Intensity']
        x = np.around(x_values).astype(int)
        new_x = list(range(1, 4001))
        old_data = {'common_column': x, 'value_column': y_values}
        old_df = pd.DataFrame(old_data)
        new_data = {'common_column': new_x}
        new_df = pd.DataFrame(new_data)
        merged_df = new_df.merge(old_df, on='common_column', how='left')
        merged_df['value_column'] = merged_df['value_column'].fillna(0)

        # Smooth the intensity values using a Gaussian kernel
        smoothed = []
        for i in merged_df['common_column'].to_list():
            kv = np.exp(-0.5 * ((merged_df['common_column'] - i) / 10) ** 2) / (10 * np.sqrt(2 * np.pi))
            smoothed.append((merged_df['value_column'] * kv).sum())

        feature_vectors_gaussian.append(smoothed)
        cas_no.append(cas_number)

    except KeyError:
        print(f"Skipping file {sdf_file} as 'IR.FREQUENCIES' data is not found.")
df_gaussian = pd.DataFrame(feature_vectors_gaussian)
df_gaussian.insert(loc=0, column='cas_no', value=cas_no)
df_gaussian.to_csv("sdf_data_gaussian.csv", index=False)


In [4]:
import os

def count_files_in_directory(directory_path):
    if not os.path.isdir(directory_path):
        raise ValueError("The provided path is not a directory.")

    file_count = 0

    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        if os.path.isfile(item_path):
            file_count += 1

    return file_count

directory_path = r"C:\Users\91830\Downloads\sdf1"
try:
    count = count_files_in_directory(directory_path)
    print(f"Number of files in '{directory_path}': {count}")
except ValueError as e:
    print(e)


Number of files in 'C:\Users\91830\Downloads\sdf1': 2364


In [7]:
import pandas as pd
import numpy as np
import os

# Directory where SDF files are located
parent_directory = r"C:\Users\91830\Downloads\sdf1"

# Get a list of SDF files in the directory
sdf_files = [file for file in os.listdir(parent_directory) if file.endswith('.sdf')]

# Initialize lists to store CAS numbers and raw feature data
cas_numbers = []
raw_features = []

# Define a function to read raw data from an SDF file
def read_raw_data(file_path):
    cas_number = os.path.basename(file_path).split(".sdf")[0]
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Initialize variables to store raw data
    raw_data = []

    # Find the start and end lines of the IR frequency data
    start_line = None
    end_line = None
    for idx, line in enumerate(lines):
        if "> <IR.FREQUENCIES>" in line:
            start_line = idx + 2  # Skip header line
        elif start_line is not None and line.strip() == '':
            end_line = idx
            break

    # Extract IR frequency data
    if start_line is not None and end_line is not None:
        for line in lines[start_line:end_line]:
            freq, intensity = map(float, line.strip().split())
            raw_data.append(intensity)  # Append raw intensity values

    return cas_number, raw_data

# Loop through each SDF file and extract raw data
for sdf_file in sdf_files:
    file_path = os.path.join(parent_directory, sdf_file)
    cas_number, raw_data = read_raw_data(file_path)
    
    if raw_data:  # Check if raw data is not empty
        cas_numbers.append(cas_number)
        raw_features.append(raw_data)

# Create a DataFrame with CAS numbers and raw features
df = pd.DataFrame({'CAS_Number': cas_numbers, 'Raw_Features': raw_features})

# Save the DataFrame to a CSV file
df.to_csv("sdf_raw_data.csv", index=False)


In [13]:
import pandas as pd
import numpy as np
import os
parent_directory = r"C:\Users\91830\Downloads\sdf1"
sdf_files = [file for file in os.listdir(parent_directory) if file.endswith('.sdf')]

# Define the fixed length for each CAS number
fixed_length = 4000
cas_numbers = []
raw_features_list = []
def read_raw_data(file_path):
    cas_number = os.path.basename(file_path).split(".sdf")[0]
    with open(file_path, 'r') as file:
        lines = file.readlines()
    raw_data = []
    start_line = None
    end_line = None
    for idx, line in enumerate(lines):
        if "> <IR.FREQUENCIES>" in line:
            start_line = idx + 2  # Skip header line
        elif start_line is not None and line.strip() == '':
            end_line = idx
            break
    if start_line is not None and end_line is not None:
        for line in lines[start_line:end_line]:
            freq, intensity = map(float, line.strip().split())
            raw_data.append(intensity)  

    return cas_number, raw_data
for sdf_file in sdf_files:
    file_path = os.path.join(parent_directory, sdf_file)
    cas_number, raw_data = read_raw_data(file_path)
    
    if raw_data:  
        cas_numbers.append(cas_number)
        
        # Truncate or zero-pad the raw data to the fixed length
        if len(raw_data) < fixed_length:
            raw_data.extend([0] * (fixed_length - len(raw_data)))
        else:
            raw_data = raw_data[:fixed_length]
        
        raw_features_list.append(raw_data)
df = pd.DataFrame({'CAS_Number': cas_numbers})
df = pd.concat([df, pd.DataFrame(raw_features_list)], axis=1)
df.to_csv("sdf_raw_data2.csv", index=False)
